In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import os
import warnings

# --- ¡Importaciones Clave para Persona 3! ---
# Usamos el Pipeline de imbalanced-learn para manejar SMOTE correctamente
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# --- Importar la función de normalización GANADORA ---
try:
    # NOTA: Usando 'pipeline_d_normalize' del ejemplo.
    # ¡Confirma que esta es la normalización ganadora que tu equipo eligió!
    from normalization_functions import pipeline_d_normalize
    print("Función 'pipeline_d_normalize' (Ganadora) importada correctamente.")
except ImportError:
    print("\n¡Error de Importación!")
    print("Asegúrate de que '1_normalization_pipelines.py' esté en la misma carpeta 'src/'.")
    exit()

ModuleNotFoundError: No module named 'cuml'

In [3]:
# --- 1. Configuración de Rutas ---
TRAIN_PATH = os.path.join("..", "data", "processed", "train.csv")
# Ruta de resultados específica para tu pipeline
RESULTS_PATH = os.path.join("..", "results", "fase_3_smote_svm_results.csv")

os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)
warnings.filterwarnings("ignore", category=UserWarning)

# --- 2. Cargar y Preparar Datos ---
print(f"Iniciando Fase 3: SMOTE + SVM + GridSearchCV")
try:
    df_train = pd.read_csv(TRAIN_PATH)
    print(f"Datos de entrenamiento cargados desde '{TRAIN_PATH}'.")
except FileNotFoundError:
    print(f"\n¡Error de Archivo! No se encontró '{TRAIN_PATH}'.")
    exit()

print("Aplicando 'pipeline_d_normalize' (Combinación Ganadora) a los datos...")
# Aplicar la normalización GANADORA
X_train = df_train['text'].apply(pipeline_d_normalize)
y_train = df_train['Polarity']
print(f"Datos normalizados listos. {len(X_train)} documentos.")

# --- 3. Definir Pipeline y Rejilla de Búsqueda ---

# Crear el Pipeline con la combinación GANADORA y tu tarea (SMOTE + SVM)
# ¡Usamos el Pipeline de imblearn!
text_pipeline = Pipeline([
    # Vectorizer Ganador: Binary (como indicaste)
    ('vectorizer', CountVectorizer(binary=True)),
    
    # Tarea Persona 3: Aplicar SMOTE 
    # random_state=0 para reproducibilidad
    ('sampler', SMOTE(random_state=0)),
    
    # Modelo Costoso: SVM 
    # random_state=0 para reproducibilidad, max_iter para evitar warnings
    ('classifier', SVC(random_state=0, max_iter=2000))
])

# Definir la rejilla de hiperparámetros para SVM
# 'classifier__' es el prefijo para aplicar los parámetros al paso 'classifier' del pipeline
param_grid = {
    # Probar dos valores de regularización (C)
    'classifier__C': [1, 10],
    
    # Probar los dos kernels más comunes
    'classifier__kernel': ['linear', 'rbf']
}

print(f"Rejilla de Hiperparámetros (Grid): {param_grid}")
print(f"Total de combinaciones a probar: {len(param_grid['classifier__C']) * len(param_grid['classifier__kernel'])} = 4")
print("Cada combinación se entrenará con 5-fold CV (Total: 20 entrenamientos de SVM + SMOTE)")

Iniciando Fase 3: SMOTE + SVM + GridSearchCV
Datos de entrenamiento cargados desde '..\data\processed\train.csv'.
Aplicando 'pipeline_d_normalize' (Combinación Ganadora) a los datos...
Datos normalizados listos. 24169 documentos.
Rejilla de Hiperparámetros (Grid): {'classifier__C': [1, 10], 'classifier__kernel': ['linear', 'rbf']}
Total de combinaciones a probar: 4 = 4
Cada combinación se entrenará con 5-fold CV (Total: 20 entrenamientos de SVM + SMOTE)


In [4]:
# --- 4. Configurar y Ejecutar GridSearchCV ---

# verbose=2 para que puedas ver el progreso y no pienses que se trabó
grid_search = GridSearchCV(
    estimator=text_pipeline,
    param_grid=param_grid,
    cv=5,               # 5 folds (como en la especificación [cite: 82])
    scoring='f1_macro', # La métrica que estamos usando [cite: 86]
    n_jobs=-1,          # ¡Usar todos los núcleos de CPU!
    verbose=2           
)

print("\n--- ¡INICIANDO GRIDSEARCHCV! ---")
print("Esto puede tardar varios minutos (SVM es más lento que MLP)...")
try:
    grid_search.fit(X_train, y_train)
except Exception as e:
    print(f"\n¡Error durante el entrenamiento de SVM! {e}")
    exit()

print("\n--- ¡GridSearchCV completado! ---")

# --- 5. Guardar y Reportar Resultados ---

best_score = grid_search.best_score_
best_params = grid_search.best_params_

print(f"Mejor F1-Macro (promedio de CV): {best_score:.4f}")
print(f"Mejores Hiperparámetros encontrados: {best_params}")

# Guardar los resultados
results_df = pd.DataFrame({
    "model": "SMOTE + SVM", # Tu tarea 
    "pipeline_base": "D (Negación) + Binary", # ¡Confirma que la normalización 'D' es la correcta!
    "best_avg_f1_macro": [best_score],
    "best_params": [str(best_params)] # Guardar como string
})

results_df.to_csv(RESULTS_PATH, index=False)
print(f"Resultados de la Fase 3 guardados en '{RESULTS_PATH}'")


--- ¡INICIANDO GRIDSEARCHCV! ---
Esto puede tardar varios minutos (SVM es más lento que MLP)...
Fitting 5 folds for each of 4 candidates, totalling 20 fits


KeyboardInterrupt: 